# Diet Recommender Chatbot - Kaggle Notebook

## Google Gen AI Intensive Course Capstone 2025Q1

This project is a part of 5-day Gen AI Intensive Course with Google

This project is a simple AI chatbot designed to provide general dietary recommendations based on user-inputted health conditions or diagnoses, acting like a friendly personal diet planner.

It utilizes Google's Gemini model for conversational AI and response generation, combined with a Retrieval-Augmented Generation (RAG) approach. A basic knowledge base of dietary information is embedded and stored in a ChromaDB vector database for efficient retrieval.

**Disclaimer:** This tool provides general information based on its knowledge base and AI generation. **It is NOT a substitute for professional medical or dietary advice.** Always consult with a qualified healthcare provider or registered dietitian for personalized recommendations specific to your health situation.

## How it Works

1. **Knowledge Base:** Sample dietary information for different conditions is defined directly within this notebook
2. **Embedding:** This knowledge base is processed, and each document is converted into a numerical vector using Google's `text-embedding-004` model.
3. **Vector Store:** These embeddings and the corresponding text documents are stored in a ChromaDB collection (in-memory for simplicity).
4. **User Query:** When the user provides their health condition, their query is also embedded.
5. **Retrieval:** ChromaDB is queried to find the documents in the knowledge base whose embeddings are most similar (semantically) to the user's query embedding.
6. **Generation:** The user's query and the retrieved documents (context) are fed into the Gemini (`gemini-2.0-flash`) model with a specific prompt instructing it to act as a meal planner and generate recommendations based _only_ on the provided context.
7. **Interaction:** The user will go through the chat interface, state (like whether a recommendation has been given), and the follow-up conversation loop.

***Note that you can't "Run All" this notebook as it requires inputs for the tasks. Run the code cell by cell from top to bottom instead to get the notebook run properly***

## 1. Setup and Installations

Install libraries.

In [1]:
!pip uninstall -qy jupyterlab
!pip install -U -q "google-genai==1.7.0"
!pip install -q -U google-generativeai chromadb pandas python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2. Import Libraries and Configure API Key

### Import libraries

In [2]:
from google import generativeai as genai_default
from google import genai
from google.genai import types

import numpy as np
import pandas as pd
import chromadb
import pandas as pd
import os
import time
from IPython.display import display, Markdown

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Set up a retry helper to "Run all" without worrying about per-minute quota.

In [3]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### Set up API key

In [4]:
# Load API Key from Kaggle Secrets
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    genai_default.configure(api_key=GOOGLE_API_KEY)
    print("API Key configured successfully.")
except Exception as e:
    print(f"Error configuring API Key from Kaggle Secrets: {e}")
    print("Please ensure you have added your GOOGLE_API_KEY as a secret in Kaggle.")
    
    # Fallback for local testing (if not on Kaggle or secret fails)
    # from dotenv import load_dotenv
    
    # load_dotenv()
    
    # GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    # if GOOGLE_API_KEY:
    #     genai.configure(api_key=GOOGLE_API_KEY)
    #     print("API Key configured from .env file (local testing).")
    # else:
    #     print("API Key not found. Please set it up either via Kaggle Secrets or a .env file.")

# Check if configuration succeeded
if not os.getenv('GOOGLE_API_KEY'):
     raise ValueError("API Key not configured. Stopping execution.")

API Key configured successfully.


## 3. Load Knowledge Base Data

We'll import the sample diet documents. 

**Disclaimer!** Note that these sample documents were previously ***one-shot***-generated using the Gemini model itself so they might contain hallucinations resulting in reccomendation inaccuracies. Therefore the reccomendations and recipes are not to be taken seriously.

Nevertheless, these samples (although limited) are supposed to serve as examples and are necessary to build the knowledge base for the RAG.

In a real scenario, this data would come from a more robust source.

In [5]:
# The documents are initially meant to be stored in a separate python file called "diet_data.py"
# Assuming diet_data.py is in the same directory or accessible path
# On Kaggle, we might need to upload diet_data.py as a utility script or copy its content here.

# Let's define the data directly here for simplicity in the Kaggle notebook environment
DIET_DOCUMENTS = [
    {
        "id": "doc1",
        "condition": "Back Pain / Inflammation",
        "text": """
        **Diet Recommendations for Back Pain & Inflammation:**

        *Focus on anti-inflammatory foods.*

        **Fruits:** Berries (blueberries, strawberries, raspberries), cherries, pineapple, oranges, apples. These are rich in antioxidants and vitamins.
        **Vegetables:** Leafy greens (spinach, kale, Swiss chard), broccoli, bell peppers, carrots, sweet potatoes. Packed with vitamins, minerals, and fiber.
        **Meats/Proteins:** Fatty fish (salmon, mackerel, sardines - high in Omega-3s), lean poultry (chicken breast, turkey), legumes (beans, lentils), tofu, nuts, and seeds (walnuts, flaxseeds, chia seeds). Limit red meat and processed meats.
        **Other:** Olive oil, turmeric, ginger, green tea. Stay well-hydrated with water. Avoid sugary drinks, refined carbohydrates (white bread, pasta), fried foods, and excessive saturated fats.

        **Simple Recipe Idea: Salmon with Roasted Veggies**
        *   Toss chopped sweet potatoes, broccoli florets, and red bell pepper strips with olive oil, salt, pepper, and a pinch of turmeric. Roast at 400°F (200°C) for 20-25 minutes.
        *   Season salmon fillets with salt, pepper, and lemon juice. Add to the baking sheet with veggies for the last 12-15 minutes of cooking, or until salmon is cooked through.
        *   Serve immediately.
        """
    },
    {
        "id": "doc2",
        "condition": "High Blood Pressure (Hypertension)",
        "text": """
        **Diet Recommendations for High Blood Pressure (Hypertension):**

        *Focus on the DASH (Dietary Approaches to Stop Hypertension) diet principles. Emphasize low sodium.*

        **Fruits:** Bananas (potassium), berries, apples, oranges, peaches, melons. Aim for 4-5 servings daily.
        **Vegetables:** Leafy greens, broccoli, carrots, tomatoes, potatoes (especially sweet potatoes). Aim for 4-5 servings daily.
        **Meats/Proteins:** Lean meats (skinless poultry, fish), beans, lentils, nuts, seeds. Limit red meat. Aim for 6 or fewer ounces per day.
        **Grains:** Whole grains (oatmeal, brown rice, whole wheat bread/pasta). Aim for 6-8 servings daily.
        **Dairy:** Low-fat or fat-free milk, yogurt, cheese. Aim for 2-3 servings daily.
        **Other:** Limit sodium intake significantly (aim for under 1500mg/day if possible, definitely under 2300mg). Reduce added sugars, saturated fats, and trans fats. Use herbs and spices for flavor instead of salt. Stay hydrated.

        **Simple Recipe Idea: Chicken and Veggie Stir-fry with Brown Rice**
        *   Cook brown rice according to package directions.
        *   Stir-fry sliced chicken breast in a little sesame or olive oil until cooked. Remove from wok/pan.
        *   Add chopped vegetables (broccoli, bell peppers, carrots, snow peas) and stir-fry until tender-crisp.
        *   Add chicken back to the pan. Toss with a low-sodium soy sauce (or coconut aminos), ginger, and garlic.
        *   Serve over brown rice.
        """
    },
    {
        "id": "doc3",
        "condition": "Type 2 Diabetes Management",
        "text": """
        **Diet Recommendations for Type 2 Diabetes Management:**

        *Focus on controlling blood sugar levels through balanced meals, portion control, and carbohydrate awareness.*

        **Fruits:** Berries, apples, pears, oranges, melon. Choose whole fruits over juices. Be mindful of portion sizes due to natural sugars.
        **Vegetables:** Non-starchy vegetables are excellent (leafy greens, broccoli, cauliflower, bell peppers, green beans, zucchini). Include starchy vegetables (potatoes, corn, peas) in moderation and count them as carbs.
        **Meats/Proteins:** Lean protein sources like fish, skinless poultry, beans, lentils, tofu, eggs, nuts in moderation. Helps with satiety and doesn't spike blood sugar significantly.
        **Grains:** Whole grains (quinoa, brown rice, oats, whole wheat bread - check labels for fiber content). Portion control is key for carbohydrate counting.
        **Dairy:** Low-fat or non-fat milk, yogurt (plain), cheese in moderation.
        **Other:** Healthy fats (avocado, olive oil, nuts, seeds). Limit sugary drinks, sweets, processed foods, white bread/rice/pasta, and trans fats. Focus on fiber-rich foods. Consistent meal timing can help manage blood sugar. Stay hydrated with water.

        **Simple Recipe Idea: Lentil Soup**
        *   Sauté chopped onions, carrots, and celery in a pot with a little olive oil.
        *   Add vegetable broth, rinsed lentils (brown or green), diced tomatoes (no salt added), and herbs like thyme or bay leaf.
        *   Simmer until lentils are tender (about 30-40 minutes).
        *   Season with black pepper. Add a splash of vinegar or lemon juice at the end if desired.
        *   This soup is high in fiber and protein.
        """
    },
    {
        "id": "doc4",
        "condition": "General Healthy Eating / Weight Management",
        "text": """
        **General Healthy Eating & Weight Management Tips:**

        *Focus on whole, unprocessed foods, portion control, and balanced macronutrients.*

        **Fruits:** Include a variety of colorful fruits daily. Good source of vitamins, minerals, and fiber.
        **Vegetables:** Fill half your plate with non-starchy vegetables at meals (leafy greens, broccoli, peppers, zucchini, etc.). They are nutrient-dense and low in calories.
        **Meats/Proteins:** Choose lean protein sources (chicken, turkey, fish, beans, lentils, tofu, eggs). Protein helps with satiety. Include protein with each meal.
        **Grains:** Opt for whole grains (brown rice, quinoa, oats, whole wheat bread) over refined grains. They provide more fiber and nutrients. Be mindful of portion sizes.
        **Fats:** Include healthy fats from sources like avocados, nuts, seeds, and olive oil. Limit saturated and trans fats found in fried foods, fatty meats, and processed snacks.
        **Other:** Drink plenty of water throughout the day. Limit sugary drinks (soda, juice) and alcohol. Reduce intake of processed foods, fast food, and high-sugar snacks. Practice mindful eating - pay attention to hunger and fullness cues. Regular physical activity is also crucial for weight management and overall health.

        **Simple Recipe Idea: Grilled Chicken Salad**
        *   Grill or pan-sear a chicken breast. Let it rest, then slice or dice it.
        *   Create a large salad base with mixed greens, spinach, cucumber, tomatoes, bell peppers, and red onion.
        *   Add the cooked chicken.
        *   Optional additions: chickpeas, a small amount of sliced avocado, or some sunflower seeds.
        *   Dress with a light vinaigrette made from olive oil, lemon juice or vinegar, and herbs.
        """
    },
    {
        "id": "doc5",
        "condition": "Iron Deficiency Anemia",
        "text": """
        **Diet Recommendations for Iron Deficiency Anemia:**
       
        *Focus on boosting iron intake and enhancing absorption.*
       
        **Fruits:** Citrus fruits (oranges, grapefruit), strawberries, bell peppers, tomatoes (Vitamin C sources eaten with iron-rich foods help absorption).
        **Vegetables:** Spinach, kale, broccoli, sweet potatoes, peas.
        **Meats/Proteins:** Lean red meat (beef, lamb), poultry (chicken, turkey - dark meat has more iron), fish (sardines, tuna), eggs, lentils, beans (kidney, chickpeas), tofu, pumpkin seeds.
        **Grains:** Iron-fortified cereals, breads, and pasta; quinoa, oats.
        **Other:** Cook in cast-iron skillets (can add small amounts of iron). Avoid drinking coffee or tea *with* iron-rich meals as tannins can inhibit absorption. Calcium supplements or high-calcium foods (like dairy) can also interfere, so time them separately.
       
        **Simple Recipe Idea: Lean Beef and Broccoli Stir-Fry**
        *   Slice lean beef thinly. Stir-fry in a wok or large pan with a little sesame oil until browned. Remove beef.
        *   Add broccoli florets and sliced red bell peppers (Vit C) to the pan, stir-fry until tender-crisp.
        *   Return beef to the pan. Add a sauce made from low-sodium soy sauce, ginger, garlic, and a touch of orange juice (Vit C).
        *   Serve immediately over brown rice or quinoa.
        """
    },
    {
        "id": "doc6",
        "condition": "GERD / Acid Reflux Management",
        "text": """
        **Diet Recommendations for GERD / Acid Reflux Management:**
        
        *Focus on avoiding common trigger foods, eating smaller meals, and choosing low-fat options.*
        
        **Fruits:** Bananas, melons (cantaloupe, honeydew), pears, apples (non-citrus, in moderation).
        **Vegetables:** Green beans, broccoli, asparagus, cauliflower, potatoes, cucumbers, leafy greens.
        **Meats/Proteins:** Lean poultry (skinless chicken/turkey - baked, grilled, or poached), fish (baked, grilled, or poached), egg whites, tofu.
        **Grains:** Oatmeal, whole-wheat bread, brown rice, couscous.
        **Dairy:** Low-fat or skim milk, low-fat yogurt (plain), small amounts of low-fat cheese (if tolerated).
        **Other:** Healthy fats in moderation (avocado, olive oil). Stay upright after eating, avoid late-night meals. Common triggers to avoid: Citrus fruits, tomatoes/tomato sauce, garlic, onions, spicy foods, fried/fatty foods, chocolate, mint, caffeine (coffee, tea, soda), alcohol, carbonated beverages.
        
        **Simple Recipe Idea: Baked Chicken Breast with Roasted Sweet Potatoes and Green Beans**
        *   Toss cubed sweet potatoes and trimmed green beans with a small amount of olive oil, salt, and pepper (avoid garlic/onion powder if sensitive).
        *   Roast on a baking sheet at 400°F (200°C) for about 20-25 minutes.
        *   Season a skinless chicken breast with salt, pepper, and perhaps some mild herbs like thyme or rosemary. Add to the baking sheet for the last 15-20 minutes of cooking, until chicken is cooked through.
        *   Serve warm.
        """
    },
    {
        "id": "doc7",
        "condition": "Constipation Relief",
        "text": """
        **Diet Recommendations for Constipation Relief:**
        
        *Focus on significantly increasing dietary fiber intake from various sources and ensuring adequate hydration.*
        
        **Fruits:** Prunes, figs, berries (raspberries, blackberries), pears (with skin), apples (with skin), kiwi, oranges.
        **Vegetables:** Broccoli, Brussels sprouts, carrots, leafy greens (spinach, kale), peas, artichokes.
        **Meats/Proteins:** Legumes are excellent fiber sources: Beans (black, kidney, pinto), lentils, chickpeas. Other proteins are generally low in fiber.
        **Grains:** Whole grains are key: Oatmeal (especially steel-cut or rolled), brown rice, quinoa, barley, whole-wheat bread and pasta, bran cereals.
        **Other:** Nuts and seeds (chia seeds, flax seeds, almonds, walnuts - in moderation), plenty of water throughout the day (aim for 8+ glasses). Regular physical activity also helps stimulate bowel movements.
        
        **Simple Recipe Idea: High-Fiber Oatmeal with Berries and Seeds**
        *   Cook rolled oats or steel-cut oats according to package directions using water or milk.
        *   Once cooked, stir in 1-2 tablespoons of chia seeds or ground flax seeds.
        *   Top with a generous handful of fresh berries (like raspberries or blueberries) and a small sprinkle of nuts if desired.
        *   Consider adding sliced prunes for an extra boost.
        """
    },
    {
        "id": "doc8",
        "condition": "Building Muscle / High Protein Diet Support",
        "text": """
        **Diet Recommendations for Building Muscle / High Protein Diet Support:**
        
        *Focus on adequate protein intake distributed throughout the day, paired with complex carbohydrates for energy and sufficient calories overall.*
        
        **Fruits:** All types are beneficial for vitamins, minerals, and antioxidants (e.g., bananas for potassium, berries for antioxidants).
        **Vegetables:** Include a wide variety for micronutrients (e.g., spinach, broccoli, sweet potatoes).
        **Meats/Proteins:** Prioritize lean sources: Chicken breast, turkey breast, lean beef (sirloin, tenderloin), fish (salmon, tuna, tilapia), eggs (whole eggs and egg whites), Greek yogurt, cottage cheese, tofu, edamame, beans, lentils.
        **Grains:** Complex carbohydrates for sustained energy: Oats, quinoa, brown rice, whole-wheat pasta, whole-wheat bread, buckwheat.
        **Dairy:** Greek yogurt, cottage cheese, milk (choose fat level based on overall calorie needs).
        **Other:** Healthy fats (avocado, nuts, seeds, olive oil). Adequate hydration is crucial. Consider protein timing around workouts (e.g., a meal or snack with protein and carbs within a couple of hours post-exercise).
        
        **Simple Recipe Idea: Grilled Chicken Breast with Quinoa and Steamed Asparagus**
        *   Cook quinoa according to package directions.
        *   Grill or pan-sear a seasoned chicken breast until cooked through.
        *   Steam or lightly sauté asparagus spears with a little olive oil, salt, and pepper.
        *   Serve the chicken breast alongside a portion of quinoa and asparagus for a balanced, high-protein meal.
        """
    },
    {
        "id": "doc9",
        "condition": "Low FODMAP Diet (Initial Phase Example for IBS Symptoms)",
        "text": """
        **Diet Recommendations for Low FODMAP Diet (Initial Phase for IBS Symptoms):**
        
        *Focus: Temporarily restrict high-FODMAP carbohydrates to identify potential IBS triggers. IMPORTANT: This diet is complex and meant to be short-term, followed by systematic reintroduction under professional guidance (doctor or registered dietitian specializing in IBS). This is just an example of *some* low-FODMAP foods.*
        
        **Fruits (Low FODMAP examples):** Unripe bananas, blueberries, cantaloupe, grapes, kiwi, lemon, lime, oranges, raspberries, strawberries (limit quantities as per Monash University app guidelines).
        **Vegetables (Low FODMAP examples):** Bell peppers (any color), carrots, cucumber, eggplant, green beans, kale, lettuce, potatoes, spinach, tomatoes, zucchini.
        **Meats/Proteins:** Plain cooked beef, chicken, fish, lamb, pork, shellfish, eggs, firm tofu.
        **Grains:** Gluten-free bread/pasta (check ingredients for high-FODMAP additives), oats, quinoa, rice (white, brown), corn tortillas.
        **Dairy:** Lactose-free milk/yogurt, hard cheeses (cheddar, mozzarella, parmesan), small amounts of butter.
        **Other:** Maple syrup (pure), peanuts, pumpkin seeds, walnuts, olive oil. Avoid: High-FODMAP fruits (apples, pears, mangoes, watermelon), vegetables (onions, garlic, cauliflower, mushrooms, asparagus), wheat, rye, beans, lentils, high-lactose dairy, honey, high-fructose corn syrup, cashews, pistachios.
        
        **Simple Recipe Idea: Simple Baked Salmon with Rice and Steamed Carrots**
        *   Cook white or brown rice according to package directions.
        *   Season a salmon fillet with salt, pepper, lemon juice, and dill (check seasoning blends for onion/garlic).
        *   Bake salmon at 400°F (200°C) for 12-15 minutes, or until cooked through.
        *   Steam sliced carrots until tender.
        *   Serve the salmon with rice and carrots. Drizzle with a little olive oil if desired.
        """
    },
    {
        "id": "doc10",
        "condition": "Heart Health (Post-Event / Prevention)",
        "text": """
        **Diet Recommendations for Heart Health (Post-Event / Prevention):**
        
        *Focus on reducing saturated and trans fats, limiting sodium, increasing fiber (especially soluble fiber), and incorporating omega-3 fatty acids.*
        **Fruits:** Berries (rich in antioxidants), apples (soluble fiber), oranges (potassium, vitamin C), bananas (potassium).
        **Vegetables:** Leafy greens (spinach, kale), broccoli, carrots, tomatoes, bell peppers. Aim for a variety of colors.
        **Meats/Proteins:** Fatty fish high in omega-3s (salmon, mackerel, herring, sardines) at least twice a week. Skinless poultry, legumes (beans, lentils, chickpeas), tofu, nuts (especially walnuts, almonds), seeds (flaxseeds, chia seeds, sunflower seeds).
        **Grains:** Whole grains like oats (contain beta-glucan, a soluble fiber), quinoa, brown rice, barley, whole-wheat bread/pasta.
        **Dairy:** Low-fat or fat-free milk, yogurt, and cheese.
        **Other:** Use olive oil instead of butter or tropical oils. Limit sodium intake by avoiding processed foods, canned soups, salty snacks, and adding salt at the table (use herbs and spices instead). Limit added sugars, sugary drinks, red meat (especially fatty cuts), and processed meats. Avoid trans fats (often listed as 'partially hydrogenated oils').
        
        **Simple Recipe Idea: Mediterranean Chickpea Salad**
        *   Rinse and drain a can of chickpeas (no salt added, if possible).
        *   Chop cucumber, tomatoes, red bell pepper, and red onion (use sparingly if preferred). Add fresh parsley.
        *   Combine chickpeas and vegetables in a bowl.
        *   Make a dressing with olive oil, lemon juice, dried oregano, and black pepper (avoid adding salt).
        *   Toss the salad with the dressing. Can be served on its own, over lettuce, or with a small portion of whole-wheat pita.
        """
    }
]

In [6]:
# Convert to DataFrame for easier handling
df = pd.DataFrame(DIET_DOCUMENTS)

print(f"Loaded {len(df)} documents.")

Loaded 10 documents.


In [7]:
display(df.head())

,id,condition,text
0,doc1,Back Pain / Inflammation,\n **Diet Recommendations for Back Pain...
1,doc2,High Blood Pressure (Hypertension),\n **Diet Recommendations for High Bloo...
2,doc3,Type 2 Diabetes Management,\n **Diet Recommendations for Type 2 Di...
3,doc4,General Healthy Eating / Weight Management,\n **General Healthy Eating & Weight Ma...
4,doc5,Iron Deficiency Anemia,\n **Diet Recommendations for Iron Defi...


## 4. Initialize models (Embedding and Generative)

We need two models: one for creating text embeddings and one for generating responses (Gemini).

List of available models.

In [8]:
for method in dir(genai_default):
    if "list" in method:
        print(method)

list_files
list_models
list_operations
list_tuned_models


In [18]:
for model in genai_default.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
m

The list above shows all the available Large Language Models (LLM) from Google. This time we will use `text-embedding-004` for text embedding and `gemini-2.0-flash` for generation task. 

In [10]:
# --- Model Initialization ---
# Using the free embedding model from Google AI
embedding_model = genai_default.GenerativeModel('models/text-embedding-004')

# Using Gemini Flash model for generation
generative_model = genai_default.GenerativeModel('gemini-2.0-flash')

print("Models initialized:")
print(f"  Embedding Model: {embedding_model.model_name}")
print(f"  Generative Model: {generative_model.model_name}")

Models initialized:
  Embedding Model: models/text-embedding-004
  Generative Model: models/gemini-2.0-flash


## 5. Create Text Embeddings

Define a function to embed text using the initialized embedding model. We'll handle potential API errors and rate limits.

In [11]:
# --- Embedding Function ---
def embed_text(text, task_type="retrieval_document"):
    """Embeds text using the Google AI embedding model."""
    try:
        # Note: Task type can be 'retrieval_query', 'retrieval_document', 'semantic_similarity', etc.
        embedding = genai_default.embed_content(model=embedding_model.model_name,
                                        content=text,
                                        task_type=task_type)
        return embedding['embedding']
    except Exception as e:
        print(f"Error embedding text: {text[:50]}... Error: {e}")
        # Implement retry logic if needed, especially for rate limits
        # time.sleep(1) # Simple backoff
        return None # Return None if embedding fails

## 6. Set up Vector Database (ChromaDB)

We'll use ChromaDB as an in-memory vector store. We'll embed our diet documents and store them.

In [12]:
# --- ChromaDB Setup ---
# Using an in-memory client for simplicity in Kaggle
client = chromadb.Client()

# Create a collection (like a table in a relational database)
DB_COLLECTION_NAME = "diet_recommendations"

try:
    # Check if collection exists, delete if it does (for clean runs in notebook)
    client.delete_collection(name=DB_COLLECTION_NAME)
    print(f"Existing collection '{DB_COLLECTION_NAME}' deleted.")
except Exception:
    print(f"Collection '{DB_COLLECTION_NAME}' does not exist, creating new one.")
    pass # Collection doesn't exist, no need to delete

# Using cosine distance as it's often good for semantic similarity
db_collection = client.create_collection(name=DB_COLLECTION_NAME, metadata={"hnsw:space": "cosine"})
print(f"Collection '{DB_COLLECTION_NAME}' created.")

# --- Embed and Store Documents ---
print("Embedding documents and adding to ChromaDB...")
doc_embeddings = []
doc_ids = []
doc_texts = []
doc_metadatas = []

for index, row in df.iterrows():
    text_to_embed = f"Condition: {row['condition']}\nDetails: {row['text']}" # Combine condition and text for embedding
    embedding = embed_text(text_to_embed)
    if embedding:
        doc_embeddings.append(embedding)
        doc_ids.append(row['id'])
        doc_texts.append(row['text']) # Store the original text for retrieval context
        doc_metadatas.append({"condition": row['condition']}) # Store condition as metadata
    else:
        print(f"Skipping document id {row['id']} due to embedding error.")
    # Add a small delay to avoid hitting potential rate limits if embedding many docs
    time.sleep(0.2)

if doc_embeddings:
    db_collection.add(
        embeddings=doc_embeddings,
        documents=doc_texts, # Store the main text content
        metadatas=doc_metadatas, # Store associated metadata
        ids=doc_ids
    )
    print(f"Successfully added {db_collection.count()} documents to the collection.")
else:
    print("No documents were successfully embedded and added.")

Collection 'diet_recommendations' does not exist, creating new one.
Collection 'diet_recommendations' created.
Embedding documents and adding to ChromaDB...
Successfully added 10 documents to the collection.


Confirm that n documents were inserted by looking at the database.

In [13]:
db_collection.count()

10

See that 10 documents were added to the vector database

Now, we can take a look at instances of the database

In [15]:
db_collection.peek(2)

{'ids': ['doc1', 'doc2'],
 'embeddings': array([[ 0.02243436, -0.00111806, -0.03931806, ...,  0.00143484,
          0.00118781, -0.00637942],
        [ 0.0159103 , -0.0198628 , -0.04006037, ...,  0.01907527,
          0.01703797, -0.03533568]]),
 'documents': ['\n        **Diet Recommendations for Back Pain & Inflammation:**\n\n        *Focus on anti-inflammatory foods.*\n\n        **Fruits:** Berries (blueberries, strawberries, raspberries), cherries, pineapple, oranges, apples. These are rich in antioxidants and vitamins.\n        **Vegetables:** Leafy greens (spinach, kale, Swiss chard), broccoli, bell peppers, carrots, sweet potatoes. Packed with vitamins, minerals, and fiber.\n        **Meats/Proteins:** Fatty fish (salmon, mackerel, sardines - high in Omega-3s), lean poultry (chicken breast, turkey), legumes (beans, lentils), tofu, nuts, and seeds (walnuts, flaxseeds, chia seeds). Limit red meat and processed meats.\n        **Other:** Olive oil, turmeric, ginger, green tea. Stay

## 7. Retrieval Function

Create a function to retrieve relevant documents from the vector database based on a user's query.

In [19]:
# --- Retrieval Function ---
def retrieve_relevant_documents(query, n_results=2):
    """Retrieves relevant documents from ChromaDB based on the query."""
    if not query:
        return []

    query_embedding = embed_text(query, task_type="retrieval_query")

    if query_embedding is None:
        print("Failed to embed query.")
        return []

    try:
        results = db_collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results,
            include=['documents', 'metadatas', 'distances'] # Include distances for potential filtering/ranking
        )
        # print(f"ChromaDB Results: {results}") # Debug print
        # Extracting documents - results['documents'] is a list containing one list of results
        retrieved_docs = results.get('documents', [[]])[0]
        return retrieved_docs
    except Exception as e:
        print(f"Error querying ChromaDB: {e}")
        return []

Retrieval function is set. We can test run it to find relevant documents

In [21]:
# Test retrieval
test_query = "I want to build muscle"

# Call the retrieval function
relevant_docs = retrieve_relevant_documents(test_query)
print(f"\nDocuments relevant to '{test_query}':")

for i, doc in enumerate(relevant_docs):
     print(f"--- Doc {i+1} ---")
     display(Markdown(doc[:500] + "...")) # Display snippet


Documents relevant to 'I want to build muscle':
--- Doc 1 ---



        **Diet Recommendations for Building Muscle / High Protein Diet Support:**
        
        *Focus on adequate protein intake distributed throughout the day, paired with complex carbohydrates for energy and sufficient calories overall.*
        
        **Fruits:** All types are beneficial for vitamins, minerals, and antioxidants (e.g., bananas for potassium, berries for antioxidants).
        **Vegetables:** Include a wide variety for micronutrients (e.g., spinach, broccoli, sweet potat...

--- Doc 2 ---



        **General Healthy Eating & Weight Management Tips:**

        *Focus on whole, unprocessed foods, portion control, and balanced macronutrients.*

        **Fruits:** Include a variety of colorful fruits daily. Good source of vitamins, minerals, and fiber.
        **Vegetables:** Fill half your plate with non-starchy vegetables at meals (leafy greens, broccoli, peppers, zucchini, etc.). They are nutrient-dense and low in calories.
        **Meats/Proteins:** Choose lean protein sources (...

See that the function is doing what it supposed to. 

After identifying a relevant text segment during the retrieval step, the next step involves crafting a generation prompt to produce a final response using the Gemini API. While this example is only limited to a single retrieved passage (due to the small dataset), in real-world applications, particularly when working with extensive datasets this process typically require retrieving multiple passages. The Gemini model will then assess which segments are pertinent to the query. This process accommodates occasional irrelevant retrievals, as the generation step automatically filters out unrelated content during the answer build-up.

## 8. Response Generation Function

This function takes the user's query and the retrieved documents, constructs a prompt for Gemini, and generates the diet recommendation response.

In [22]:
# --- Response Generation Function ---
def generate_response(user_problem, retrieved_docs):
    """Generates a diet recommendation response using Gemini and retrieved context."""
    if not user_problem:
        return "Please tell me what health condition you'd like diet recommendations for."

    context = "\n\n---\n\n".join(retrieved_docs) if retrieved_docs else "No specific context found."

    prompt = f"""You are a friendly and helpful AI assistant acting like a personal diet planner. Your goal is to provide diet recommendations based *only* on the provided context information.

    **User's Problem:** "{user_problem}"

    **Context Information:**
    ```
    {context}
    ```

    **Instructions:**
    1. Carefully review the context information related to the user's problem.
    2. If relevant context is found, synthesize the information to provide clear diet recommendations. List specific suggestions for:
        * Fruits
        * Vegetables
        * Meats/Proteins
        * Other relevant foods/tips
        * Suggest 1-2 simple recipe ideas mentioned or inspired by the context.
    3. If no relevant context is found or the context doesn't match the user's problem well, state that you couldn't find specific information for that exact condition in your current knowledge base, but you can offer general healthy eating tips if they like. Do NOT invent recommendations if context is missing.
    4. Keep the tone friendly, empathetic, and encouraging, like a family meal planner.
    5. **Crucially:** After providing the recommendations (or stating you couldn't find specific info), ALWAYS end your response by asking: "Do you want to discuss further details about your problem? (yes/no)"

    **Your Response:**
    """

    # print("--- Sending Prompt to Gemini ---") # Debug
    # print(prompt) # Debug
    # print("-------------------------------") # Debug

    try:
        response = generative_model.generate_content(prompt)
        # print("--- Gemini Raw Response ---") # Debug
        # print(response) # Debug
        # print("--------------------------") # Debug
        return response.text
    except Exception as e:
        print(f"Error generating response from Gemini: {e}")
        # Check for specific errors like blocked content if applicable
        # print(f"Candidate: {response.candidates}")
        return "Sorry, I encountered an error while generating the response. Please try again."

The function is all set. Now here's the interesting part. We will try to interact with the model

## 9. Main Chat Interaction Loop

This loop handles the conversation flow: getting the initial problem, providing recommendations, and handling follow-up discussion.

In [23]:
# --- Main Chat Loop ---

print("AI Diet Reccomender: Hi there! I'm your friendly meal planning assistant.")
print("AI Diet Reccomender: I can help suggest some foods based on health conditions.")

# Get initial problem
initial_problem = input("AI Diet Reccomender: What's your problem or health condition? (e.g., 'back pain', 'high blood pressure')\nUser: ")

if initial_problem.strip():
    # 1. Retrieve relevant documents
    retrieved_docs = retrieve_relevant_documents(initial_problem)

    # 2. Generate initial response
    initial_response = generate_response(initial_problem, retrieved_docs)
    display(Markdown(f"**AI Diet Reccomender:**\n{initial_response}")) # Use Markdown for better formatting

    # 3. Handle follow-up conversation
    while True:
        follow_up_decision = input("User (yes/no): ").strip().lower()

        if follow_up_decision == "yes":
            further_input = input("AI Diet Reccomender: Okay, what else is on your mind regarding this? Or what details would you like to discuss?\nUser: ")

            if further_input.strip():
                # Construct a conversational prompt for follow-up
                # We can reuse retrieval or just let Gemini handle conversationally
                # For simplicity here, let Gemini continue based on the new input + initial context

                follow_up_prompt = f"""You are a friendly AI personal diet planner continuing a conversation.

                The user's initial problem was: "{initial_problem}"
                They previously received some recommendations and now want to discuss further.

                Their latest input is: "{further_input}"

                Instructions:
                1. Respond helpfully and conversationally to the user's latest input, keeping the initial problem in mind.
                2. Provide additional details, clarification, or answer related questions based on your general knowledge and the context of the initial problem (but prioritize safety and avoid giving medical advice).
                3. Keep the tone friendly and supportive.
                4. **Crucially:** After your response, ALWAYS ask: "Do you want to continue discussing? (yes/no)"

                **Your Response:**
                """

                try:
                    follow_up_response = generative_model.generate_content(follow_up_prompt)
                    display(Markdown(f"**AI Diet Reccomender:**\n{follow_up_response.text}"))
                except Exception as e:
                    print(f"Error generating follow-up response: {e}")
                    print("AI Diet Reccomender: Sorry, I had trouble processing that. Do you want to try asking differently? (yes/no)")


            else:
                print("AI Diet Reccomender: It seems like you didn't type anything. Do you want to continue discussing? (yes/no)")

        elif follow_up_decision == "no":
            print("AI Diet Reccomender: Okay, sounds good! Remember, these are just general suggestions. Always consult with a healthcare professional or registered dietitian for personalized medical and dietary advice. Stay healthy!")
            break
        else:
            print("AI Diet Reccomender: Sorry, I didn't quite catch that. Please type 'yes' to continue or 'no' to stop.")

else:
    print("AI Diet Reccomender: It looks like you didn't specify a problem. If you change your mind, just run me again!")

AI Diet Reccomender: Hi there! I'm your friendly meal planning assistant.
AI Diet Reccomender: I can help suggest some foods based on health conditions.


AI Diet Reccomender: What's your problem or health condition? (e.g., 'back pain', 'high blood pressure')
User:  My blood pressure is currently high, bro


**AI Diet Reccomender:**
Hey bro! I'm sorry to hear about your high blood pressure, but I'm here to help with some diet recommendations! Based on the information I have, focusing on the DASH diet principles with low sodium will be beneficial. Here's what I suggest:

*   **Fruits:** Load up on bananas (for potassium), berries, apples, oranges, peaches, and melons. Aim for 4-5 servings each day.
*   **Vegetables:** Go for leafy greens, broccoli, carrots, tomatoes, and potatoes, especially sweet potatoes. Try to get 4-5 servings daily.
*   **Meats/Proteins:** Choose lean meats like skinless poultry and fish. Beans, lentils, nuts, and seeds are also great choices. Limit red meat to less than 6 ounces a day.
*   **Grains:** Opt for whole grains like oatmeal, brown rice, and whole wheat bread/pasta. Aim for 6-8 servings daily.
*   **Dairy:** Stick to low-fat or fat-free milk, yogurt, and cheese. Try to get 2-3 servings each day.
*   **Other:** This is important – limit your sodium intake significantly, aiming for under 1500mg a day if you can, but definitely under 2300mg. Cut back on added sugars, saturated fats, and trans fats. Use herbs and spices to flavor your food instead of salt. And don't forget to stay hydrated!

Here's a simple recipe idea to get you started:

**Chicken and Veggie Stir-fry with Brown Rice**

*   Cook brown rice according to package directions.
*   Stir-fry sliced chicken breast in a little sesame or olive oil until cooked. Remove from wok/pan.
*   Add chopped vegetables (broccoli, bell peppers, carrots, snow peas) and stir-fry until tender-crisp.
*   Add chicken back to the pan. Toss with a low-sodium soy sauce (or coconut aminos), ginger, and garlic.
*   Serve over brown rice.

Do you want to discuss further details about your problem? (yes/no)


User (yes/no):  yes
AI Diet Reccomender: Okay, what else is on your mind regarding this? Or what details would you like to discuss?
User:  But I want to build muscle as well, bro


**AI Diet Reccomender:**
Alright, bro! I hear you. High blood pressure *and* wanting to build muscle – that's a solid goal! It definitely adds a layer of complexity, but it's totally achievable.

Here's the thing: we need to be smart about how we approach this. Building muscle often involves increased protein intake and potentially more sodium depending on your food choices and workout intensity. Both of those can impact blood pressure.

So, here's the plan:

*   **Prioritize Lean Protein Sources:** Think chicken breast, fish (salmon is great!), lean beef, turkey, and plant-based options like beans, lentils, tofu, and edamame. These are lower in saturated fat and can help you hit your protein goals without negatively impacting your blood pressure as much as, say, a super fatty cut of steak.
*   **Watch the Sodium:** Pay close attention to sodium content in your protein sources (especially processed meats like deli slices). Season your food with herbs and spices instead of relying heavily on salt.
*   **Don't Neglect the Potassium:** Potassium helps counter the effects of sodium. Load up on potassium-rich foods like bananas, sweet potatoes, spinach, and avocados. They're also great sources of energy for your workouts.
*   **Hydrate Properly:** Staying well-hydrated is crucial for both blood pressure and muscle growth. Aim for plenty of water throughout the day.
*   **Resistance Training is Key (but carefully!)**: Resistance training is fantastic for building muscle. However, remember to breathe properly during your sets. Holding your breath can spike your blood pressure. Work with a reasonable weight and prioritize proper form. It may also be worth discussing resistance training with your doctor to make sure you're cleared to continue/begin safely.
*   **Monitor your Blood Pressure:** Keep tabs on your blood pressure to see how your diet and training are affecting it. This will help you make adjustments as needed.

We can definitely dive deeper into specific meal plans and workout strategies that consider both your blood pressure and muscle-building goals. We could talk about timing your meals, choosing the right supplements (if any), and finding low-sodium protein powders if you use them. Remember to also consult with your doctor and a registered dietician or certified personal trainer for personalized advice! They can help you create a safe and effective plan based on your individual needs and medical history.

Do you want to continue discussing? (yes/no)


...

AI Diet Reccomender: Sorry, I didn't quite catch that. Please type 'yes' to continue or 'no' to stop.


User (yes/no):  yes
AI Diet Reccomender: Okay, what else is on your mind regarding this? Or what details would you like to discuss?
User:  I just want to eat healthy in general


**AI Diet Reccomender:**
Okay, I understand. You want to focus on eating healthy in general, and that's a fantastic goal, especially considering your high blood pressure! Eating healthy can definitely have a positive impact on managing blood pressure and improving your overall well-being.

Since you want to eat healthy in general, let's broaden our focus a bit. Besides limiting sodium and increasing potassium (which are great for blood pressure), we can think about a balanced diet that includes plenty of fruits, vegetables, whole grains, lean protein, and healthy fats.

Here are some general tips for healthy eating, keeping your blood pressure in mind:

*   **Focus on Whole Foods:** Base your diet around foods that are as close to their natural state as possible. Think fresh fruits and vegetables, whole grains like brown rice and quinoa, and lean proteins like chicken breast or fish.
*   **Read Food Labels Carefully:** Pay attention to serving sizes, calories, fat, sodium, and added sugars. This will help you make informed choices and avoid hidden sources of sodium.
*   **Cook at Home More Often:** This gives you more control over ingredients and portion sizes. Restaurant meals are often higher in sodium, fat, and calories.
*   **Hydrate Well:** Drink plenty of water throughout the day. Sometimes thirst can be mistaken for hunger.
*   **Portion Control:** Be mindful of how much you're eating at each meal. Using smaller plates can help.
*   **Limit Processed Foods:** These are often high in sodium, unhealthy fats, and added sugars.

Remember, small changes can make a big difference over time. Don't feel like you have to overhaul your entire diet overnight. Start with one or two goals and gradually incorporate more healthy habits.

Do you want to continue discussing? (yes/no)


User (yes/no):  no


AI Diet Reccomender: Okay, sounds good! Remember, these are just general suggestions. Always consult with a healthcare professional or registered dietitian for personalized medical and dietary advice. Stay healthy!


## Limitations & Future Improvements

- Limited Knowledge Base: The sample data in diet_data.py is very basic. A real application needs a comprehensive, medically validated database.
- No Real-time Data: Doesn't access external, up-to-the-minute medical information.
- Basic RAG: Retrieval is based on simple similarity. More advanced techniques could improve relevance.
- No User Profiles: Doesn't store user history, allergies, or preferences across sessions.
- Safety: While basic prompts guide the AI, more robust safety layers and content filtering would be needed for a production app.
- Error Handling: Basic error handling is included, but could be more comprehensive.

This setup gives us a functional prototype. Again, **It is NOT a substitute for professional medical or dietary advice.**
Play around and tinker with your own version of this RAG by copying this notebook. Let me know if you have any questions or suggestions for improvements!

## Contact

- [LinkedIn](https://www.linkedin.com/in/robert-modalo-78b97580)
- [Medium](https://medium.com/@robertmodalo2)
- [GitHub](https://github.com/robertmodalocodes)